In [9]:
from flask import Flask
from flask import request
from flask import jsonify
import json
import os
import time
from fairseq.models.roberta import RobertaModel
from google.cloud import storage
app = Flask(__name__)
import cloudstorage
from google.cloud import storage
import shutil
import wget

In [36]:
CLOUD_STORAGE_BUCKET = os.environ['CLOUD_STORAGE_BUCKET']

In [ ]:
url1 = 'https://storage.googleapis.com/salesken-models/ROBERTa-STS-B/checkpoint_best.pt'
filename1 = wget.download(url1)

url2 = 'https://storage.googleapis.com/salesken-models/ROBERTa-STS-B/STS-B-bin.tar.gz'
filename2 = wget.download(url2)
shutil.unpack_archive(filename2, '/home/vmagent/app')

In [56]:
roberta = RobertaModel.from_pretrained(
    '/home/vmagent/app/',
    checkpoint_file=filename1,
    data_name_or_path='/home/vmagent/app/STS-B-bin'
)

In [55]:
roberta.eval()

In [6]:
@app.route("/sentence_similarity_roberta",methods=["POST","GET"])
def sentence_similarity_roberta():
    r = []
    result = []
    
    list1=request.form.getlist('sentence1')
    list2=request.form.getlist('sentence2')
    list1 = list1[0].replace("\"",'').replace("[",'').replace("]",'')
    s1 = list1.split(",")
    list2 = list2[0].replace("\"",'').replace("[",'').replace("]",'')
    s2 = list2.split(",")
    
    for i in s1:
        for j in s2:
            tokens = roberta.encode(i, j)
            features = roberta.extract_features(tokens)
            predictions = roberta.model.classification_heads['sentence_classification_head'](features)
            r.append({"sentence1":i, 'sentence2':j, 'roberta_score':str(predictions.item())})
        result.append(r)
        r = []
    return jsonify(result)

In [ ]:
if __name__ == '__main__':
    app.run(debug=True, threaded=True, host='0.0.0.0', port='5010')